## Heilbrigðislíkan
Smá byrjun

In [30]:
import simpy as sp
import streamlit as st
import numpy as np
import math
import random
import time
import os
import signal
from multiprocessing import Process
import pandas as pd

In [31]:
class Manneskja:
    def __init__(self,aldur):
        self.aldur = aldur
        if self.aldur < 65:
            self.prob = 0.02
        else:
            self.prob = 0.1
    def MtoP(self):
        p = Patient(self.aldur,"móttaka")
        return p

class Patient:
    def __init__(self,aldur,place):
        self.aldur = aldur
        self.place = place
        if place == "skurðaaðgerð":
            self.p_death = 0.1
            self.p_surgery = 0.0
            self.p_mottaka = 0.9
        if place == "móttaka":
            self.p_death = 0.0
            self.p_home = 0.7
            self.p_surgery = 0.3
    def PtoM(self):
        m = Manneskja(self.aldur)
        return m

In [32]:
class Spitali:
    def __init__(self,cap):
        self.cap = cap
        self.amount = 0
    def addP(self):
        if self.amount == self.cap:
            raise Exception("Nú er spítalinn fullur það tók")
        self.amount += 1
    def removeP(self):
        self.amount -= 1

In [39]:
def sim():
    N = 1000  # fjöldi fólks
    t = 1  # tímaskref
    timi = 0  # byrjunartími
    P = []  # fjöldi sjúklinga á spítala
    M = []  # fjöldi manneskja ekki á spítala
    S = Spitali(500)  # spítali með capacity 500
    data = {}
    data[0] = 0
    chart = st.line_chart(pd.DataFrame([data]))
    for p in range(0, 299):
        age = random.randint(65, 99)
        pi = Manneskja(age)
        M.append(pi)
    for p in range(300, N):
        age = random.randint(1, 64)
        pi = Manneskja(age)
        M.append(pi)
    STOP = 10000
    while(timi < STOP):
        timi += t
        for m in M:
            r = random.random()
            if r < m.prob:
                S.addP()
                pi = m.MtoP()
                M.remove(m)
                P.append(pi)
                data[timi] = data[timi-t] + 1
        for p in P:
            r = random.random()
            if p.place == "móttaka":
                if r < p.p_death:
                    P.remove(p)
                    data[timi] = data[timi-t] -1
                    S.removeP()
                if r >= p.p_death and r <= p.p_surgery:
                    P.remove(p)
                    data[timi] = data[timi-t] -1
                    S.removeP()
                    m = p.PtoM()
                    M.append(m)
                else:
                    p.place = "skurðaaðgerð"
            if p.place == "skurðaaðgerð":
                if r < p.p_surgery:
                    p.place = "skurðaaðgerð"
                if r >= p.p_surgery and r <= p.p_death:
                    P.remove(p)
                    data[timi] = data[timi-t] -1
                    S.removeP()
                else:
                    p.place = "móttaka"
        #df = pd.DataFrame([data])
        chart.add_rows([data[timi]])
        time.sleep(0.1)
        print("fjöldi fólks á spítala á degi",timi,"er",S.amount)


In [41]:
st.sidebar.title("Controls")
start = st.sidebar.button("Start")
stop = st.sidebar.button("Stop")

state = st.session_state.get("pid", None)  # Use string "pid" as the key

if start:
    p = Process(target=sim)
    p.start()
    state = {"pid": p.pid}  # Assign the "pid" value to the state dictionary
    st.write("Started process with pid:", state["pid"])

if stop:
    if state and "pid" in state:
        os.kill(state["pid"], signal.SIGKILL)
        st.write("Stopped process with pid:", state["pid"])
        state["pid"] = None
    else:
        st.write("No process running.")

st.session_state["pid"] = state  # Store the updated state back to session_state